In [95]:
##############
#Dependencies#
##############
%matplotlib inline

import pandas
import gmaps
from config import (gkey, 
#                     route)
                   )

In [96]:
gmaps.configure(api_key=gkey)

In [169]:
route = ['S567','S583','S568','S582','S569','S570','S581',
'S580','S571','S579','S572','S578','S573',"S577","S587"]
station_df = pandas.read_csv('data/station_config.csv')
route_df= pandas.read_csv('data/route_summary.csv')
station_df = station_df.set_index("Station")
route_df = route_df.set_index("Station")
route_df.head()

,Station.1,Time,Occupancy,Speed,Flow
Station,,,,,
S567,NaN,Wed Oct 03 01:16:59 CDT 2018,0.31,54,NaN
S567,NaN,Wed Oct 03 01:16:59 CDT 2018,0.31,54,NaN
S567,NaN,Sat Oct 06 19:03:29 CDT 2018,3.54,67,520
S567,NaN,Sat Oct 06 19:03:29 CDT 2018,3.54,67,520
S567,NaN,Sat Oct 06 19:32:29 CDT 2018,5.91,65,840


In [193]:

Results = {}


locations = []

for station in route:
        Flow =  route_df.loc[station,'Flow'].tail(1)[0]
        Speed = route_df.loc[station,'Speed'].tail(1)[0]

        Max_Flow = route_df.loc[station,'Flow'].dropna()
        Max_Flow = max(Max_Flow.values)
        Max_Speed = route_df.loc[station,'Speed'].dropna()
        Max_Speed = max(Max_Speed.values)
        
        lat, lon = station_df.loc[station, ['Lat', 'Lng']]
        locations.append((lat, lon))

        Results[station] = {
           "Current": {"Speed" : Speed,"Flow" : Flow},
           "Max": {"Speed" : Speed, "Flow" : Flow},
            "Lat":lat, 
            "Lon":lon}

In [194]:
Results

{'S567': {'Current': {'Speed': '61', 'Flow': '880'},
  'Max': {'Speed': '61', 'Flow': '880'},
  'Lat': 44.974470000000004,
  'Lon': -93.2487},
 'S583': {'Current': {'Speed': '64', 'Flow': '630'},
  'Max': {'Speed': '64', 'Flow': '630'},
  'Lat': 44.98175,
  'Lon': -93.24396999999999},
 'S568': {'Current': {'Speed': '47', 'Flow': '720'},
  'Max': {'Speed': '47', 'Flow': '720'},
  'Lat': 44.983990000000006,
  'Lon': -93.24197},
 'S582': {'Current': {'Speed': '81', 'Flow': '1160'},
  'Max': {'Speed': '81', 'Flow': '1160'},
  'Lat': 44.99115,
  'Lon': -93.23659},
 'S569': {'Current': {'Speed': '75', 'Flow': '780'},
  'Max': {'Speed': '75', 'Flow': '780'},
  'Lat': 44.99116,
  'Lon': -93.23629},
 'S570': {'Current': {'Speed': '76', 'Flow': '600'},
  'Max': {'Speed': '76', 'Flow': '600'},
  'Lat': 44.99644,
  'Lon': -93.23823},
 'S581': {'Current': {'Speed': '69', 'Flow': '1040'},
  'Max': {'Speed': '69', 'Flow': '1040'},
  'Lat': 44.99646,
  'Lon': -93.23853000000001},
 'S580': {'Current': 

In [196]:
# Create bank symbol layer
fig = gmaps.figure()

station_layer = gmaps.symbol_layer(locations)

fig.add_layer(station_layer)

fig

Figure(layout=FigureLayout(height='420px'))

array(['520', '520', '840', '840', '840', '1000', '800', '880', '1080',
       '1040', '680', '1160', '960', '960', '880', '880', '880', '480',
       '1080', '1080', '1080', '1160', '920', '800', '1240', '1240',
       '440', '1040', '840', '840', '720', '640', '600', '600', '760',
       '1200', '480', '480', '680', '920', '800', '920', '1040', '720',
       '1160', '720', '640', '360', '680', '840', '840', '640', '760',
       '760', '760', '1160', '1160', '520', '520', '520', '840', '840',
       '840', '1000', '800', '880', '1080', '1040', '680', '1160', '960',
       '960', '880', '880', '880', '480', '1080', '1080', '1080', '1160',
       '920', '800', '1240', '1240', '440', '1040', '840', '840', '720',
       '640', '600', '600', '760', '1200', '480', '480', '680', '920',
       '800', '920', '1040', '720', '1160', '720', '640', '360', '680',
       '840', '840', '640', '760', '760', '760', '1160', '1160', '520',
       '520', '520', '520', '840', '840', '840', '1000', '800', '